In [1]:
import os
import warnings
import pandas as pd
import matplotlib.pyplot as plt
from typing import List
from dotenv import load_dotenv
from tqdm.auto import tqdm

load_dotenv()
warnings.filterwarnings('ignore')

AWS_KEY = os.getenv('AWS_ACCESS_KEY')
AWS_SECRET_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_S3_BUCKET = os.getenv('AWS_S3_BUCKET')
REVIEWS_FILE_TEMPLATE = os.path.join('reviews', 'reviews_partition_{}.parquet')

storage_options = {
    'key': AWS_KEY,
    'secret': AWS_SECRET_KEY
}

reviews_files = [
    os.path.join(
        's3://',
        AWS_S3_BUCKET,
        REVIEWS_FILE_TEMPLATE.format(partition + 1)
    )
    for partition in range(20)
]
metadata_file = os.path.join(
    's3://',
    AWS_S3_BUCKET,
    'metadata/metadata.json'
)
metadata_proc_file = os.path.join(
    's3://',
    AWS_S3_BUCKET,
    'metadata/metadata_features.json'
)

%load_ext autoreload
%autoreload 2

In [3]:
metadata = pd.read_json(metadata_proc_file, storage_options=storage_options, orient='index')
metadata = metadata.astype({'release_date': 'datetime64[D]'})
metadata.head()

FSTimeoutError: 

In [821]:
load_params = {
    'column': ['id', 'rating', 'author'],
    'storage_options': storage_options
}

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
def load_reviews(partition_path: List[str], **load_params) -> pd.DataFrame:
    reviews_partitions = []
    for path in tqdm(partition_path):
        reviews_partition = pd.read_parquet(path, **load_params)
        reviews_partitions.append(reviews_partition)
        break
    reviews = pd.concat(reviews_partitions)
    return reviews

In [834]:
def create_interaction_matrix(dataset: pd.DataFrame) -> pd.DataFrame:
    matrix = (
        dataset
        [['id', 'author', 'rating']]
        .drop_duplicates()
        .pivot(index='author', columns='id', values='rating')
        .astype('float32')
    )
    return matrix

In [835]:
interaction_matrix = create_interaction_matrix(reviews)

In [867]:
reviews_left_cnt = 10
cnt = (
    reviews
    .groupby('author')
    .agg({'id': 'count'})
    .sort_values('id', ascending=False)
    .reset_index()
    .query(f'id >= {reviews_left_cnt}')
    .drop(columns='id')
)
cnt

,author
0,/user/ur2898520/
1,/user/ur0482513/
2,/user/ur5876717/
3,/user/ur2467618/
4,/user/ur2488512/
...,...
1494,/user/ur1217890/
1495,/user/ur35717158/
1496,/user/ur0793218/
1497,/user/ur23843983/


In [849]:
reviews.shape

(204818, 9)

In [873]:
reviews['id'].nunique()

1191

In [ ]:
usecols = ['id', 'rating', 'author']

In [868]:
reviews_ = pd.merge(reviews, cnt, how='inner', on='author')
reviews_

,index,id,text,rating,title,author,upvotes,total_votes,review_date
0,0,/title/tt0113142/,I liked this movie the best out of all the '90...,9.0,A modern day classic kaijyu movie,/user/ur3225430/,24,26,2005-10-10
1,940,/title/tt0111161/,"After a sobering experience in the prison, wha...",10.0,The Best Movie Ever Made !?,/user/ur3225430/,1,3,2011-10-05
2,83,/title/tt0106912/,If a bunch of guys comes into town claiming th...,5.0,One of the most intriguing story ever told - ...,/user/ur3225430/,0,0,2011-09-04
3,372,/title/tt1099212/,I saw other Twilight saga first so I'm biased ...,NaN,Best Vampire Story Ever !!,/user/ur3225430/,3,8,2011-10-08
4,401,/title/tt0144084/,"Of all the psycho killer movies I've seen, thi...",7.0,Best psycho killer movie,/user/ur3225430/,0,0,2005-12-21
...,...,...,...,...,...,...,...,...,...
42602,821,/title/tt0167404/,This summer I went and saw the awesome movie t...,NaN,The spectator says...Best Movie EVER!!!,/user/ur0443828/,0,0,1999-09-17
42603,822,/title/tt0167404/,"The Sixth Sense was an awesome movie, and by f...",NaN,Kellie says The Sixth Sense is best movie of ...,/user/ur0443828/,0,0,1999-09-17
42604,823,/title/tt0167404/,This summer I saw the movie The Sixth Sense. I...,NaN,"Megan thinks it is a scary, disturbing drama!",/user/ur0443828/,0,0,1999-09-17
42605,824,/title/tt0167404/,Let me tell you something. This summer am I gl...,NaN,A Great Suspense film that will leave you on ...,/user/ur0443828/,0,0,1999-09-17


In [869]:
interaction_matrix_ = create_interaction_matrix(reviews_)

In [870]:
def check_sparsity(interaction_matrix: pd.DataFrame) -> float:
    num_users, num_titles = interaction_matrix.shape
    num_total_elements = num_users * num_titles
    num_nonempty_elements = (~interaction_matrix.isna()).sum().sum()
    sparsity = 1 - num_nonempty_elements / num_total_elements
    return 100 * sparsity

In [ ]:
def train_test_split(interaction_matrix: pd.DataFrame, test_fraction: float) -> float:


In [876]:
interaction_matrix_

id,/title/tt0012532/,/title/tt0014341/,/title/tt0015174/,/title/tt0015324/,/title/tt0015841/,/title/tt0017765/,/title/tt0019585/,/title/tt0020018/,/title/tt0020641/,/title/tt0021106/,...,/title/tt9359344/,/title/tt9430780/,/title/tt9526784/,/title/tt9592496/,/title/tt9637132/,/title/tt9689696/,/title/tt9738784/,/title/tt9742794/,/title/tt9806336/,/title/tt9907782/
author,,,,,,,,,,,,,,,,,,,,,
/user/ur0001220/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/user/ur0002746/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/user/ur0003136/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/user/ur0007613/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/user/ur0011762/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/user/ur9938570/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/user/ur9972457/,NaN,8.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/user/ur99782462/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [894]:
np.isnan(x).values

array([ True,  True,  True, ...,  True,  True,  True])

In [898]:
interaction_matrix_.values

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, 10., nan, ..., nan, nan, nan]], dtype=float32)

In [902]:
interaction_matrix_.shape

(1499, 1131)

In [901]:
for i in interaction_matrix_.values:
    print(i, i.shape)
    break

[nan nan nan ... nan nan nan] (1131,)


In [930]:
type(nn.flatten()[0])

numpy.int64

In [939]:
p = 0.2
size = int(p * len(nn))
a = np.random.choice(nn.flatten(), size=size)
a

array([564, 237, 194])

In [937]:
np.setdiff1d(nn, a)

array([ 74, 194, 237, 301, 391, 511, 519, 564, 581, 590, 664, 738, 810])

In [912]:
nn

array([[ 74],
       [172],
       [194],
       [214],
       [237],
       [301],
       [391],
       [511],
       [519],
       [564],
       [581],
       [590],
       [664],
       [738],
       [810]])

In [920]:
x[nn.flatten()]

array([7., 9., 5., 9., 7., 8., 8., 8., 8., 6., 3., 8., 9., 8., 9.],
      dtype=float32)

In [921]:
np.put(x, nn.flatten(), True)

In [923]:
x[nn.flatten()]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)

In [916]:
np.where(x == nn, True, False)

(15, 1131)

In [911]:
nn = np.argwhere(~np.isnan(x))
np.whereinteraction_matrix_.values[0, nn]

AttributeError: module 'numpy' has no attribute 'whereinteraction_matrix_'

In [909]:
np.random.choice(nn.flatten(), )

564

In [910]:
np.random.choice?

Docstring:
choice(a, size=None, replace=True, p=None)

Generates a random sample from a given 1-D array

.. versionadded:: 1.7.0

.. note::
    New code should use the ``choice`` method of a ``default_rng()``
    instance instead; please see the :ref:`random-quick-start`.

Parameters
----------
a : 1-D array-like or int
    If an ndarray, a random sample is generated from its elements.
    If an int, the random sample is generated as if it were ``np.arange(a)``
size : int or tuple of ints, optional
    Output shape.  If the given shape is, e.g., ``(m, n, k)``, then
    ``m * n * k`` samples are drawn.  Default is None, in which case a
    single value is returned.
replace : boolean, optional
    Whether the sample is with or without replacement. Default is True,
    meaning that a value of ``a`` can be selected multiple times.
p : 1-D array-like, optional
    The probabilities associated with each entry in a.
    If not given, the sample assumes a uniform distribution over all
    entr

In [942]:
p = 0.2
test_idxs = []
test_vals = []
train_idxs = []
train_vals = []
for user_ratings in interaction_matrix_.values:
    nonnull_idx = np.argwhere(~np.isnan(user_ratings)).flatten()
    size = int(p * len(nonnull_idx))
    test_idx = np.random.choice(nonnull_idx, size=size)
    train_idx = np.setdiff1d(nonnull_idx, test_idx)
    test_idxs.append(test_idx)
    train_idxs.append(train_idx)
    break
# x = interaction_matrix_.iloc[0].values
# np.argwhere(~np.isnan(x))

In [948]:
user_ratings[~train_idxs[0]]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
      dtype=float32)

In [949]:
np.where(train_idxs[0], True, False)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [944]:
train_idxs

[array([ 74, 194, 214, 237, 301, 511, 519, 564, 581, 590, 664, 810])]

In [887]:
interaction_matrix_.groupby(level=0).apply(np.array).apply()

author
/user/ur0001220/     [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...
/user/ur0002746/     [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...
/user/ur0003136/     [[nan, nan, nan, nan, nan, nan, nan, nan, 4.0,...
/user/ur0007613/     [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...
/user/ur0011762/     [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...
                                           ...                        
/user/ur9938570/     [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...
/user/ur9972457/     [[nan, 8.0, nan, 9.0, nan, nan, nan, nan, nan,...
/user/ur99782462/    [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...
/user/ur99809306/    [[nan, nan, nan, nan, nan, nan, nan, nan, nan,...
/user/ur9987799/     [[nan, 10.0, nan, 10.0, nan, 8.0, nan, nan, na...
Length: 1499, dtype: object

In [ ]:
import numpy as np

indices = np.random.choice(np.arange(myarray.size), replace=False,
                           size=int(myarray.size * 0.2))


In [875]:
interaction_matrix_.isna()

id,/title/tt0012532/,/title/tt0014341/,/title/tt0015174/,/title/tt0015324/,/title/tt0015841/,/title/tt0017765/,/title/tt0019585/,/title/tt0020018/,/title/tt0020641/,/title/tt0021106/,...,/title/tt9359344/,/title/tt9430780/,/title/tt9526784/,/title/tt9592496/,/title/tt9637132/,/title/tt9689696/,/title/tt9738784/,/title/tt9742794/,/title/tt9806336/,/title/tt9907782/
author,,,,,,,,,,,,,,,,,,,,,
/user/ur0001220/,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
/user/ur0002746/,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
/user/ur0003136/,True,True,True,True,True,True,True,True,False,True,...,True,True,True,True,True,True,True,True,True,True
/user/ur0007613/,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
/user/ur0011762/,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/user/ur9938570/,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
/user/ur9972457/,True,False,True,False,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
/user/ur99782462/,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [877]:
interaction_matrix_.shape

(1499, 1131)

In [865]:
interaction_matrix.shape

(130264, 1191)

In [872]:
check_sparsity(interaction_matrix_)

97.75429419790028

In [863]:
check_sparsity(interaction_matrix)

99.88075755787395

In [860]:
interaction_matrix_

id,/title/tt0012532/,/title/tt0014341/,/title/tt0015174/,/title/tt0015324/,/title/tt0015841/,/title/tt0017765/,/title/tt0019585/,/title/tt0020018/,/title/tt0020641/,/title/tt0021106/,...,/title/tt9359344/,/title/tt9430780/,/title/tt9526784/,/title/tt9592496/,/title/tt9637132/,/title/tt9689696/,/title/tt9738784/,/title/tt9742794/,/title/tt9806336/,/title/tt9907782/
author,,,,,,,,,,,,,,,,,,,,,
/user/ur0001220/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/user/ur0002746/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/user/ur0003136/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/user/ur0003696/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/user/ur0006042/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/user/ur99784917/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/user/ur99809306/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/user/ur99877336/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [830]:
reviews[['id', 'author', 'rating']].pivot(columns='id', index='author', values='rating')

ValueError: Index contains duplicate entries, cannot reshape

In [6]:
user_item_matrix = (
    reviews
    [['id', 'author', 'rating']]
    .drop_duplicates()
    .pivot(index='author', columns='id', values='rating')
)
memusage = user_item_matrix.memory_usage(deep=True).sum() / 1024 / 1024
print(f'{memusage:.2f} Mb')

601.00 Mb


In [10]:
reviews

,index,id,text,rating,title,author,upvotes,total_votes,review_date
0,0,/title/tt0113142/,I liked this movie the best out of all the '90...,9.0,A modern day classic kaijyu movie,/user/ur3225430/,24,26,2005-10-10
1,1,/title/tt0113142/,When Gamera first appeared in Japanese theater...,10.0,a magnificently entertaining monster movie; t...,/user/ur6321000/,13,13,2009-11-23
2,2,/title/tt0113142/,"Sooner than I expected, GAMERA Trilogy COMPLET...",9.0,Didn't Disappoint !!,/user/ur20916867/,10,10,2009-04-09
3,3,/title/tt0113142/,This film is the first of the newer Gamera mon...,8.0,Good Introduction to Revive Gamera Trilogy.,/user/ur0437174/,6,6,2007-06-13
4,4,/title/tt0113142/,"This is a good, definitely 90ish monster movie...",8.0,good 90ish monster movie,/user/ur5156469/,7,7,2005-12-04
...,...,...,...,...,...,...,...,...,...
204813,53,/title/tt0118691/,The Nanny is a great sitcom and I love it. Bu...,NaN,The Nanny goes to Eastern Europe,/user/ur0398112/,0,2,1999-07-28
204814,54,/title/tt0118691/,Go into this movie for what it is and it's hum...,10.0,Forgot how funny this movie actually was,/user/ur119571184/,0,1,2020-06-10
204815,55,/title/tt0118691/,"Seen some poor reviews, so what, I love it to ...",10.0,Love it,/user/ur136147672/,0,0,2021-10-23
204816,56,/title/tt0118691/,Drescher has an excellent performance!!! And D...,8.0,Very kind movie!!!,/user/ur19378453/,0,0,2022-01-29


In [11]:
num_users, num_titles = user_item_matrix.shape
num_total_elements = num_users * num_titles
num_nonempty_elements = (~user_item_matrix.isna()).sum().sum()
sparsity = 100 * (num_total_elements - num_nonempty_elements) / num_total_elements
print(sparsity)

99.88075755787395


In [ ]:
memusage_f32 = user_item_matrix.memory_usage(deep=True).sum() / 1024 / 1024
memusage_f16 = user_item_matrix.astype('float16').memory_usage(deep=True).sum() / 1024 / 1024
print(
    f'Float32 memory footprint: {memusage_f32:.1f} Mb',
    f'Float16 memory footprint: {memusage_f16:.1f} Mb',
    sep='\n'
)

In [23]:
vals = metadata.nlargest(20, 'rating').index
relevant_metadata = metadata[metadata.index.isin(vals)]
relevant_metadata

,main_genre,original_title,poster_url,actors,imdb_recommendations,reviews_collected_flg,rating,num_votes,user_review_num,critic_review_num,...,original_language,production_company_1,production_company_2,production_company_3,filming_location,filming_country,budget,boxoffice_gross_domestic,boxoffice_gross_opening,boxoffice_gross_worldwide
/title/tt0050083/,Crime,12 Angry Men,https://m.media-amazon.com/images/M/MV5BMWU4N2...,"{'1': '/name/nm0000020', '2': '/name/nm0002011...","{'1': '/title/tt0108052/', '2': '/title/tt0111...",1,9.0,771000,1900.0,152.0,...,English,Orion-Nova Productions,None,None,"New York County Courthouse - 60 Centre Street,...",USA,"$350,000 (estimated)",None,None,$955
/title/tt0068646/,Crime,The Godfather,https://m.media-amazon.com/images/M/MV5BM2MyNj...,"{'1': '/name/nm0000008', '2': '/name/nm0000199...","{'1': '/title/tt0071562/', '2': '/title/tt0111...",1,9.2,1800000,5000.0,197.0,...,English,Paramount Pictures,Albert S. Ruddy Productions,Alfran Productions,"Forza d'Agrò, Messina, Sicily, Italy",Italy,"$6,000,000 (estimated)","$136,381,073","$302,393","$250,341,816"
/title/tt0071562/,Crime,The Godfather: Part II,https://m.media-amazon.com/images/M/MV5BMWMwMG...,"{'1': '/name/nm0000199', '2': '/name/nm0000134...","{'1': '/title/tt0068646/', '2': '/title/tt0099...",1,9.0,1200000,1200.0,187.0,...,English,Paramount Pictures,The Coppola Company,American Zoetrope,"Kaiser Estate, 4000 W Lake Blvd, Homewood, Lak...",USA,"$13,000,000 (estimated)","$47,834,595","$171,417","$47,961,919"
/title/tt0108052/,Biography,Schindler's List,https://m.media-amazon.com/images/M/MV5BNDE4OT...,"{'1': '/name/nm0000553', '2': '/name/nm0000146...","{'1': '/title/tt0110912/', '2': '/title/tt0111...",1,9.0,1300000,2100.0,173.0,...,English,Universal Pictures,Amblin Entertainment,None,"Auschwitz-Birkenau Concentration Camp, Oswieci...",Poland,"$22,000,000 (estimated)","$96,898,818","$656,636","$322,161,245"
/title/tt0111161/,Drama,The Shawshank Redemption,https://m.media-amazon.com/images/M/MV5BMDFkYT...,"{'1': '/name/nm0000209', '2': '/name/nm0000151...","{'1': '/title/tt0468569/', '2': '/title/tt0109...",1,9.3,2600000,10200.0,164.0,...,English,Castle Rock Entertainment,None,None,"127A Smithfield Road, Frederiksted, Virgin Isl...",Islands,"$25,000,000 (estimated)","$28,767,189","$727,327","$28,884,504"
/title/tt0167260/,Action,The Lord of the Rings: The Return of the King,https://m.media-amazon.com/images/M/MV5BNzA5ZD...,"{'1': '/name/nm0000704', '2': '/name/nm0001557...","{'1': '/title/tt0167261/', '2': '/title/tt0120...",1,9.0,1800000,4000.0,342.0,...,English,New Line Cinema,WingNut Films,The Saul Zaentz Company,"Hinuera Valley, Matamata, Waikato, New Zealand",Zealand,"$94,000,000 (estimated)","$378,251,207","$72,629,713","$1,146,436,214"
/title/tt0232152/,Drama,Nagara Haavu,https://m.media-amazon.com/images/M/MV5BMTEyYW...,"{'1': '/name/nm0024259', '2': '/name/nm0033175...","{'1': '/title/tt1339248/', '2': '/title/tt0315...",1,9.0,1100,10.0,1.0,...,Kannada,Sri Eswari Productions,None,None,None,None,None,None,None,None
/title/tt0249795/,Comedy,Mayabazar,https://m.media-amazon.com/images/M/MV5BMmQwNj...,"{'1': '/name/nm0004417', '2': '/name/nm0710036...","{'1': '/title/tt0311594/', '2': '/title/tt0249...",1,9.1,5000,34.0,1.0,...,Telugu,Vijaya Pictures,None,None,"Vauhini Studios, Chennai, Tamil Nadu, India",India,None,None,None,None
/title/tt0252487/,Comedy,The Chaos Class,https://m.media-amazon.com/images/M/MV5BOWI4NG...,"{'1': '/name/nm0839017', '2': '/name/nm0654805...","{'1': '/title/tt0252488/', '2': '/title/tt0252...",1,9.3,40000,72.0,1.0,...,Turkish,Arzu Film,None,None,"Camlica, Istanbul, Turkey",Turkey,None,None,None,None
/title/tt0259534/,Animation,Ramayana: The Legend of Prince Rama,https://m.media-amazon.com/images/M/MV5BOTk4NG...,"{'1': '/name/nm4468244', '2': '/name/nm1336047...","{'1': '/title/tt0488836/', '2': '/title/tt1674...",1,9.1,4800,29.0,NaN,...,Hindi,Nippon Ramayana Film Co.,TEM,None,None,None,None,None,None,None


In [34]:
relevant_metadata[['genre_1', 'genre_2', 'genre_3']].apply(lambda x: ', '.join(x) if x else '', axis=1)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [24]:
relevant_metadata.columns

Index(['main_genre', 'original_title', 'poster_url', 'actors',
       'imdb_recommendations', 'reviews_collected_flg', 'rating', 'num_votes',
       'user_review_num', 'critic_review_num', 'metascore', 'genre_1',
       'genre_2', 'genre_3', 'release_date', 'also_known_as', 'runtime',
       'country_of_origin_1', 'country_of_origin_2', 'country_of_origin_3',
       'original_language', 'production_company_1', 'production_company_2',
       'production_company_3', 'filming_location', 'filming_country', 'budget',
       'boxoffice_gross_domestic', 'boxoffice_gross_opening',
       'boxoffice_gross_worldwide'],
      dtype='object')

In [43]:
metadata['release_date'].dt.strftime('%B %Y')

/title/tt0002130/        July 1911
/title/tt0003740/         May 1914
/title/tt0004635/    February 1914
/title/tt0004707/    December 1914
/title/tt0004972/       March 1915
                         ...      
/title/tt9898858/       April 2020
/title/tt9900782/     October 2019
/title/tt9907782/    February 2022
/title/tt9911196/    February 2020
/title/tt9916362/     October 2020
Name: release_date, Length: 23583, dtype: object

In [36]:
metadata['genre_1']

/title/tt0002130/    Adventure
/title/tt0003740/    Adventure
/title/tt0004635/       Action
/title/tt0004707/       Comedy
/title/tt0004972/        Drama
                       ...    
/title/tt9898858/       Action
/title/tt9900782/       Action
/title/tt9907782/      Fantasy
/title/tt9911196/       Comedy
/title/tt9916362/        Drama
Name: genre_1, Length: 23583, dtype: object

In [ ]:
relevant_metadata[['original_title', 'poster_url', 'rating', '']]

In [204]:
from abc import ABCMeta
from typing import List, Dict, Any
from itertools import chain


class BaseRecommender(metaclass=ABCMeta):
    RELEVANT_METADATA = [
        'original_title',
        'genre',
        'release_date',
        'country_of_origin_1',
        'production_company_1',
        'rating',
        'poster_url'
    ]
    def __init__(self, metadata: pd.DataFrame, **model_params):
        self.metadata = metadata

    def _recommend(self, **user_input) -> Dict[str, float]:
        pass

    def _format_recommendations(
        self,
        identifiers: Dict[str, float]
    ) -> Dict[str, Any]:
        """
        Returns relevant metadata for each predicted movie.
        """
        metadata_ = self.metadata[self.metadata.index.isin(identifiers)]
        metadata_.loc[identifiers.keys(), 'relevance'] = list(identifiers.values())
        metadata_['genre'] = (
            metadata_[['genre_1', 'genre_2', 'genre_3']]
            .apply(lambda x: ', '.join(x.dropna()), axis=1)
        )
        metadata_['release_date'] = (
            metadata_['release_date']
            .dt.strftime('%B %Y')
        )
        metadata_['rating'] = metadata_['rating'].round(1)

        return (
            metadata_
            .sort_values('relevance', ascending=False)
            [self.RELEVANT_METADATA]
            .to_dict(orient='records')
        )
    
    def recommend(
        self,
        user_liked_movies: List[str] = None,
        user_preferences: Dict[str, Any] = None
    ) -> Dict[str, Any]:
        user_input = {
            'user_liked_movies': user_liked_movies,
            'user_preferences': user_preferences
        }
        raw_recommendations = self._recommend(**user_input)
        return self._format_recommendations(raw_recommendations)


class PopularRecommender(BaseRecommender):
    def __init__(self, metadata: pd.DataFrame, top_n: int):
        BaseRecommender.__init__(self, metadata)
        self.popular_titles = (
            self.metadata
            .query('num_votes > 50000')
            .nlargest(top_n, 'rating')
        )

    def _recommend(self, **user_input) -> Dict[str, float]:
        return self.popular_titles['rating'].to_dict()


class IMDBRecommender(BaseRecommender):
    def __init__(self, metadata: pd.DataFrame, top_n: int):
        BaseRecommender.__init__(self, metadata)
        self.metadata['imdb_recommendations_set'] = (
            self.metadata['imdb_recommendations']
            .apply(lambda x: set(x.values()))
        )
        self.top_n = top_n
        
    def _recommend(self, user_liked_movies: List[str], **user_input)\
            -> List[str]:
        # metadata about movies that user liked
        user_liked_movies_metadata = self.metadata[
            self.metadata.index.isin(user_liked_movies)
        ]
        # set of recommendations for each liked movie
        all_imdb_recommendations = (
            user_liked_movies_metadata['imdb_recommendations_set']
            .tolist()
        )
        # metadata for unique recommendations for all liked movies
        # recommend only movies with highest rating
        unique_recommendations_metadata = self.metadata[
            self.metadata.index.isin(set(chain(*all_imdb_recommendations)))
        ]
        return (
            unique_recommendations_metadata
            .nlargest(self.top_n, 'rating')
            ['rating']
            .to_dict()
        )

In [206]:
liked_movies = np.random.choice(metadata.index, 5).tolist()

In [209]:
%%time
poprec = PopularRecommender(metadata, 10)
poprec.recommend()[:2]

CPU times: user 15 ms, sys: 674 µs, total: 15.7 ms
Wall time: 15.1 ms


[{'original_title': 'The Shawshank Redemption',
  'genre': 'Drama',
  'release_date': 'October 1994',
  'country_of_origin_1': 'United States',
  'production_company_1': 'Castle Rock Entertainment',
  'rating': 9.3,
  'poster_url': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_QL75_UX190_CR0,0,190,281_.jpg'},
 {'original_title': 'The Godfather',
  'genre': 'Crime, Drama',
  'release_date': 'March 1972',
  'country_of_origin_1': 'United States',
  'production_company_1': 'Paramount Pictures',
  'rating': 9.2,
  'poster_url': 'https://m.media-amazon.com/images/M/MV5BM2MyNjYxNmUtYTAwNi00MTYxLWJmNWYtYzZlODY3ZTk3OTFlXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_QL75_UY281_CR4,0,190,281_.jpg'}]

In [208]:
%%time
imdbrec = IMDBRecommender(metadata, 10)
imdbrec.recommend(liked_movies)[:2]

CPU times: user 52.1 ms, sys: 1.17 ms, total: 53.2 ms
Wall time: 52.5 ms


[{'original_title': 'Rocky',
  'genre': 'Drama, Sport',
  'release_date': 'December 1976',
  'country_of_origin_1': 'United States',
  'production_company_1': 'Chartoff-Winkler Productions',
  'rating': 8.1,
  'poster_url': 'https://m.media-amazon.com/images/M/MV5BNTBkMjg2MjYtYTZjOS00ODQ0LTg0MDEtM2FiNmJmOGU1NGEwXkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_QL75_UX190_CR0,4,190,281_.jpg'},
 {'original_title': 'First Blood',
  'genre': 'Action, Adventure, Thriller',
  'release_date': 'October 1982',
  'country_of_origin_1': 'United States',
  'production_company_1': 'Anabasis N.V.',
  'rating': 7.7,
  'poster_url': 'https://m.media-amazon.com/images/M/MV5BODBmOWU2YWMtZGUzZi00YzRhLWJjNDAtYTUwNWVkNDcyZmU5XkEyXkFqcGdeQXVyNDk3NzU2MTQ@._V1_QL75_UX190_CR0,2,190,281_.jpg'}]

In [211]:
metadata.columns

Index(['main_genre', 'original_title', 'poster_url', 'actors',
       'imdb_recommendations', 'reviews_collected_flg', 'rating', 'num_votes',
       'user_review_num', 'critic_review_num', 'metascore', 'genre_1',
       'genre_2', 'genre_3', 'release_date', 'also_known_as', 'runtime',
       'country_of_origin_1', 'country_of_origin_2', 'country_of_origin_3',
       'original_language', 'production_company_1', 'production_company_2',
       'production_company_3', 'filming_location', 'filming_country', 'budget',
       'boxoffice_gross_domestic', 'boxoffice_gross_opening',
       'boxoffice_gross_worldwide', 'imdb_recommendations_set'],
      dtype='object')

In [218]:
metadata.groupby('original_language')['main_genre'].count().sort_values(ascending=False)[:10]

original_language
English     15589
French       1122
Hindi        1066
Japanese      633
Italian       515
Spanish       500
Turkish       452
German        368
Tamil         268
Korean        244
Name: main_genre, dtype: int64

In [212]:
metadata['genre_1']

/title/tt0002130/    Adventure
/title/tt0003740/    Adventure
/title/tt0004635/       Action
/title/tt0004707/       Comedy
/title/tt0004972/        Drama
                       ...    
/title/tt9898858/       Action
/title/tt9900782/       Action
/title/tt9907782/      Fantasy
/title/tt9911196/       Comedy
/title/tt9916362/        Drama
Name: genre_1, Length: 23583, dtype: object

In [3]:
top_n_actors = 5
actors = [
    (list(item.values()) + ['']*(top_n_actors - len(item)))[:top_n_actors]
    for item in metadata['actors']
]
actors_df = pd.DataFrame.from_records(actors)

In [5]:
actors_df

,0,1,2,3,4
0,/name/nm0660139,/name/nm0685283,/name/nm0209738,/name/nm3942815,/name/nm1375863
1,/name/nm0021935,/name/nm0702894,/name/nm0656034,/name/nm0146028,/name/nm0544842
2,/name/nm0267914,/name/nm0758457,/name/nm0455612,/name/nm0277317,/name/nm0298243
3,/name/nm0000122,/name/nm0237597,/name/nm0635667,/name/nm0841501,/name/nm0071658
4,/name/nm0001273,/name/nm0550615,/name/nm0910400,/name/nm0178270,/name/nm0017488
...,...,...,...,...,...
23578,/name/nm1159180,/name/nm0378245,/name/nm10067359,/name/nm2365811,/name/nm0498165
23579,/name/nm1912683,/name/nm1230844,/name/nm10183124,/name/nm6998719,/name/nm6649988
23580,/name/nm2933542,/name/nm0717709,/name/nm0677944,/name/nm3646923,/name/nm0079451
23581,/name/nm0277932,/name/nm0824373,/name/nm3362584,/name/nm2558112,/name/nm10877188


In [7]:
from sklearn.preprocessing import OneHotEncoder

In [38]:
main_actors_categories = 20
actor_encoder = OneHotEncoder(max_categories=main_actors_categories)
actor_encoder.fit(actors_df[0].to_frame())

OneHotEncoder(max_categories=20)

In [56]:
def get_actor_categories(
    actors: pd.Series,
    max_categories: int,
    prefix: str = ''
) -> pd.DataFrame:
    actor_encoder = OneHotEncoder(max_categories=max_categories)
    actor_encoder.fit(actors.to_frame())

    actor_encoded = pd.DataFrame(
        data=actor_encoder.transform(actors.to_frame()).todense(),
        columns=actor_encoder.get_feature_names(),
        dtype=bool
    )
    actor_encoded.columns = actor_encoded.columns.str.removeprefix('x0_')
    return actor_encoded.add_prefix(prefix)

get_actor_categories(actors_df[0], 20, 'first_')

,first_/name/nm0000044,first_/name/nm0000078,first_/name/nm0000115,first_/name/nm0000134,first_/name/nm0000142,first_/name/nm0000158,first_/name/nm0000230,first_/name/nm0000243,first_/name/nm0000246,first_/name/nm0000323,first_/name/nm0000329,first_/name/nm0000821,first_/name/nm0001191,first_/name/nm0006795,first_/name/nm0222426,first_/name/nm0451321,first_/name/nm0474774,first_/name/nm0482320,first_/name/nm0839017,first_infrequent_sklearn
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23578,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
23579,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
23580,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
23581,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [43]:
actor_encoder.get_feature_names()

array(['x0_/name/nm0000044', 'x0_/name/nm0000078', 'x0_/name/nm0000115',
       'x0_/name/nm0000134', 'x0_/name/nm0000142', 'x0_/name/nm0000158',
       'x0_/name/nm0000230', 'x0_/name/nm0000243', 'x0_/name/nm0000246',
       'x0_/name/nm0000323', 'x0_/name/nm0000329', 'x0_/name/nm0000821',
       'x0_/name/nm0001191', 'x0_/name/nm0006795', 'x0_/name/nm0222426',
       'x0_/name/nm0451321', 'x0_/name/nm0474774', 'x0_/name/nm0482320',
       'x0_/name/nm0839017', 'x0_infrequent_sklearn'], dtype=object)

In [57]:
metadata.columns

Index(['main_genre', 'original_title', 'poster_url', 'actors',
       'imdb_recommendations', 'reviews_collected_flg', 'rating', 'num_votes',
       'user_review_num', 'critic_review_num', 'metascore', 'genre_1',
       'genre_2', 'genre_3', 'release_date', 'also_known_as', 'runtime',
       'country_of_origin_1', 'country_of_origin_2', 'country_of_origin_3',
       'original_language', 'production_company_1', 'production_company_2',
       'production_company_3', 'filming_location', 'filming_country', 'budget',
       'boxoffice_gross_domestic', 'boxoffice_gross_opening',
       'boxoffice_gross_worldwide'],
      dtype='object')

In [745]:
from typing import List, Any


def find_boxoffice_quantile_range(x, q):
    for i, j, l, r in zip(q, q[1:], q.index, q.index[1:]):
        if i <= x < j:
            return f'boxoffice_from_q{l}_to_q{r}'
    return 'boxoffice_unknown_q'


def concat_genres(row):
    genre = [row['genre_1']]
    if row['genre_2']:
        genre += [row['genre_2']]
    if row['genre_3']:
        genre += [row['genre_3']]
    return ', '.join(sorted(genre))


def get_runtime_bin_labels(bins: List[float]) -> List[str]:
    return [
        f'runtime_from_q{100*l:.0f}_to_q{100*r:.0f}'
        for l, r in zip(bins, bins[1:])
    ]



TransformerHyperparams = {
    'n_actor_feat': 10,
    'n_genre_feat': 20,
    'n_prod_comp_feat': 20,
    'n_country_feat': 20,
    'n_lang_feat': 20,
    'boxoffice_quantiles': [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1],
    'runtime_quantiles': [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1],
    'rating_bins': [0, 4, 6, 8, 10]
}

class SimpleFeatureTransformer:
    def __init__(
        self,
        n_actor_feat: int,
        n_genre_feat: int,
        n_prod_comp_feat: int,
        n_country_feat: int,
        n_lang_feat: int,
        boxoffice_quantiles: List[float],
        runtime_quantiles: List[float],
        rating_bins: List[int]
    ):
        self.n_actor_feat = n_actor_feat
        self.n_genre_feat = n_genre_feat
        self.n_prod_comp_feat = n_prod_comp_feat
        self.n_country_feat = n_country_feat
        self.n_lang_feat = n_lang_feat
        self.boxoffice_quantiles = boxoffice_quantiles
        self.runtime_quantiles = runtime_quantiles
        self.rating_bins = rating_bins

    @staticmethod
    def get_genre_features(
        metadata: pd.DataFrame,
        n_categories: int
    ) -> pd.DataFrame:
        genre_comb = (
            metadata[['genre_1', 'genre_2', 'genre_3']]
            .apply(concat_genres, axis=1)
            .to_frame()
        )
        encoder = OneHotEncoder(max_categories=n_categories)
        encoder.fit(genre_comb)
        features = pd.DataFrame(
            data=encoder.transform(genre_comb).todense(),
            index=metadata.index,
            columns=encoder.get_feature_names()
        )
        features.columns = features.columns.str.removeprefix('x0_')
        return features

    @staticmethod
    def get_release_decade_features(metadata: pd.DataFrame) -> pd.DataFrame:
        release_decade = (
            metadata['release_date']
            .dt.year.round(-1)
            .to_frame()
        )
        encoder = OneHotEncoder()
        encoder.fit(release_decade)
        features = pd.DataFrame(
            data=encoder.transform(release_decade).todense(),
            index=metadata.index,
            columns=encoder.get_feature_names()
        )
        features.columns = features.columns.str.removeprefix('x0_')
        features.columns = features.columns.str[:-2]
        features = features.add_prefix('released_in_').add_suffix('s')
        features.columns.values[-1] = 'unknown_release_decade'
        return features.astype('int8')

    @staticmethod
    def get_actor_features(
        metadata: pd.DataFrame,
        n_categories: int
    ) -> pd.DataFrame:
        top_n = 3
        actors = [
            (list(item.values()) + ['']*(top_n - len(item)))[:top_n]
            for item in metadata['actors']
        ]
        actors = pd.DataFrame.from_records(actors, index=metadata.index)

        encoder = OneHotEncoder(max_categories=n_categories)

        features = []
        for i in range(top_n):
            encoder.fit(actors[i].to_frame())
            feat = pd.DataFrame(
                data=encoder.transform(actors[i].to_frame()).todense(),
                index=metadata.index,
                columns=encoder.get_feature_names()
            ).drop('x0_infrequent_sklearn', axis=1)
            features.append(feat)

        features = pd.concat(features, axis=1)
        features = features[features.columns.unique()]
        features['another'] = features.sum(axis=1) == 0
        features.columns = features.columns.str.removeprefix('x0_/name/')
        return (
            features
            .astype('int8')
            .add_prefix('actor_')
            .drop(columns='actor_x0_', errors='ignore')
        )
        
    @staticmethod
    def get_country_features(
        metadata: pd.DataFrame,
        n_categories: int
    ) -> pd.DataFrame:
        country = metadata['country_of_origin_1'].to_frame()
        encoder = OneHotEncoder(max_categories=n_categories)
        encoder.fit(country)
        features = pd.DataFrame(
            data=encoder.transform(country).todense(),
            index=metadata.index,
            columns=encoder.get_feature_names()
        )
        features.columns = features.columns.str.removeprefix('x0_')
        features.columns.values[-1] = 'infrequent_country'
        return features.add_prefix('originated_in_').astype('int8')

    @staticmethod
    def get_language_features(metadata: pd.DataFrame, n_categories: int) -> pd.DataFrame:
        language = metadata['original_language'].to_frame()
        encoder = OneHotEncoder(max_categories=n_categories)
        encoder.fit(language)
        features = pd.DataFrame(
            data=encoder.transform(language).todense(),
            index=metadata.index,
            columns=encoder.get_feature_names()
        )
        features.columns = features.columns.str.removeprefix('x0_')
        features.columns.values[-1] = 'infrequent'
        return features.add_suffix('_language').astype('int8')

    @staticmethod
    def get_rating_features(metadata: pd.DataFrame, bins: List[int]) -> pd.DataFrame:
        rating_cat_mapping = {
            f'({l}, {r}]': f'rating_from_{l}_to_{r}'
            for l, r in zip(bins, bins[1:])
        }
        rating = (
            pd.cut(metadata['rating'], bins)
            .astype(str)
            .apply(lambda x: rating_cat_mapping.get(x))
            .to_frame()
        )
        encoder = OneHotEncoder()
        encoder.fit(rating)
        features = pd.DataFrame(
            data=encoder.transform(rating).todense(),
            index=metadata.index,
            columns=encoder.get_feature_names()
        )
        features.columns = features.columns.str.removeprefix('x0_')
        return features.astype('int8')

    @staticmethod
    def get_prod_company_features(metadata: pd.DataFrame, n_categories: int) -> pd.DataFrame:
        company = metadata['production_company_1'].to_frame()
        encoder = OneHotEncoder(max_categories=n_categories)
        encoder.fit(company)
        features = pd.DataFrame(
            data=encoder.transform(company).todense(),
            index=metadata.index,
            columns=encoder.get_feature_names()
        )
        features.columns = features.columns.str.removeprefix('x0_')
        features.columns.values[-1] = 'infrequent_company'
        if 'None' in features.columns:
            features = features.rename({'None': 'unknown_company'}, axis=1)
        return features.add_prefix('produced_by_').astype('int8')        

    @staticmethod
    def get_boxoffice_features(
        metadata: pd.DataFrame,
        quantiles: List[float]
    ) -> pd.DataFrame:
        dollars = metadata[metadata['boxoffice_gross_worldwide'].str[0] == '$']
        dollars['decade'] = dollars['release_date'].dt.year.round(-1)
        dollars['boxoffice'] = (
            dollars['boxoffice_gross_worldwide']
            .str.replace('$', '')
            .str.replace(',', '')
            .astype('int')
        )
        dollars_pivot = (
            dollars
            .groupby('decade')
            ['boxoffice']
            .quantile(quantiles, interpolation='nearest')
            .reset_index()
            .assign(quantile=lambda x: (100*x['level_1']).astype(int))
            .pivot(index='decade', columns='quantile', values='boxoffice')
        )
        dollars_pivot[100] += 1
        
        boxoffice_cat = (
            dollars[['boxoffice', 'decade']]
            .apply(lambda x: find_boxoffice_quantile_range(
                x['boxoffice'], dollars_pivot.loc[x['decade']]
            ), axis=1)
        )
        boxoffice_cat.name = 'boxoffice_category'
        unknown_boxoffice_cat = (
            metadata[~metadata.index.isin(boxoffice_cat.index)]
            .assign(boxoffice_category='boxoffice_unknown_q')
            ['boxoffice_category']
        )
        boxoffice = pd.concat([boxoffice_cat, unknown_boxoffice_cat])
        return pd.get_dummies(boxoffice)

    @staticmethod
    def get_runtime_features(metadata, quantiles: List[float]) -> pd.DataFrame:
        labels = get_runtime_bin_labels(quantiles)
        runtime_features = pd.qcut(metadata['runtime'], quantiles, labels=labels)
        return pd.get_dummies(runtime_features)

    def transform(self, metadata: pd.DataFrame) -> pd.DataFrame:
        metadata_ = metadata.copy(deep=False)
        features = pd.concat([
            self.get_boxoffice_features(metadata_, self.boxoffice_quantiles),
            self.get_actor_features(metadata_, self.n_actor_feat),
            self.get_country_features(metadata_, self.n_country_feat),
            self.get_genre_features(metadata_, self.n_genre_feat),
            self.get_language_features(metadata_, self.n_lang_feat),
            self.get_prod_company_features(metadata_, self.n_prod_comp_feat),
            self.get_rating_features(metadata_, self.rating_bins),
            self.get_release_decade_features(metadata_),
            self.get_runtime_features(metadata_, self.runtime_quantiles)
        ], axis=1)
        return features.astype('uint8')

In [751]:
threshold = 5_000
metadata_truncated = metadata[metadata['num_votes'] > threshold]
feats = SimpleFeatureTransformer(**TransformerHyperparams).transform(metadata_truncated)
feats.shape

(14502, 136)

In [752]:
%%time
feat_mat = linear_kernel(feats)

CPU times: user 3.36 s, sys: 921 ms, total: 4.28 s
Wall time: 1.45 s


In [754]:
feat_mat

array([[9., 4., 6., ..., 2., 3., 2.],
       [4., 9., 4., ..., 4., 4., 4.],
       [6., 4., 9., ..., 2., 3., 2.],
       ...,
       [2., 4., 2., ..., 9., 8., 7.],
       [3., 4., 3., ..., 8., 9., 6.],
       [2., 4., 2., ..., 7., 6., 9.]])

In [749]:
metadata_truncated.shape

(14502, 30)

In [683]:
%%time
feats_mat = feats.dot(feats.T)
feats_mat.shape

CPU times: user 8.07 s, sys: 51 ms, total: 8.12 s
Wall time: 8.12 s


(9530, 9530)

In [702]:
%%time
cos_feats_mat = cosine_similarity(feats)
cos_feats_mat = pd.DataFrame(cos_feats_mat, index=feats.index, columns=feats.index)
cos_feats_mat.shape

CPU times: user 1.82 s, sys: 614 ms, total: 2.44 s
Wall time: 519 ms


(9530, 9530)

In [777]:
metadata_truncated[metadata_truncated['original_title'] == 'Toy Story']

,main_genre,original_title,poster_url,actors,imdb_recommendations,reviews_collected_flg,rating,num_votes,user_review_num,critic_review_num,...,original_language,production_company_1,production_company_2,production_company_3,filming_location,filming_country,budget,boxoffice_gross_domestic,boxoffice_gross_opening,boxoffice_gross_worldwide
/title/tt0114709/,Animation,Toy Story,https://m.media-amazon.com/images/M/MV5BMDU2ZW...,"{'1': '/name/nm0000158', '2': '/name/nm0000741...","{'1': '/title/tt0435761/', '2': '/title/tt0120...",1,8.3,973000,742.0,159.0,...,English,Walt Disney Pictures,Pixar Animation Studios,None,None,None,"$30,000,000 (estimated)","$223,225,679","$29,140,617","$394,436,586"


In [778]:
titles = [
    '/title/tt7286456/',
    '/title/tt0172495/',
    '/title/tt0468569/',
    '/title/tt0068646/',
    '/title/tt0114709/'
]
titles

['/title/tt7286456/',
 '/title/tt0172495/',
 '/title/tt0468569/',
 '/title/tt0068646/',
 '/title/tt0114709/']

In [787]:
%%time
cos_feats_mat.loc[set(cos_feats_mat.index) - set(titles)]

CPU times: user 91.6 ms, sys: 125 ms, total: 217 ms
Wall time: 245 ms


,/title/tt0010323/,/title/tt0012349/,/title/tt0013442/,/title/tt0015648/,/title/tt0015864/,/title/tt0016220/,/title/tt0017075/,/title/tt0017136/,/title/tt0018455/,/title/tt0018578/,...,/title/tt9701940/,/title/tt9701942/,/title/tt9777644/,/title/tt9783600/,/title/tt9784456/,/title/tt9845564/,/title/tt9848626/,/title/tt9860728/,/title/tt9866072/,/title/tt9898858/
/title/tt0278488/,0.333333,0.666667,0.555556,0.444444,0.777778,0.555556,0.555556,0.333333,0.444444,0.666667,...,0.666667,0.666667,0.666667,0.444444,0.333333,0.444444,0.333333,0.444444,0.555556,0.444444
/title/tt0108037/,0.222222,0.555556,0.333333,0.333333,0.555556,0.444444,0.333333,0.222222,0.222222,0.555556,...,0.555556,0.555556,0.555556,0.444444,0.222222,0.444444,0.333333,0.444444,0.555556,0.333333
/title/tt0762114/,0.111111,0.444444,0.222222,0.111111,0.555556,0.333333,0.333333,0.111111,0.333333,0.333333,...,0.333333,0.333333,0.333333,0.444444,0.444444,0.111111,0.444444,0.555556,0.444444,0.444444
/title/tt1416801/,0.444444,0.555556,0.444444,0.444444,0.555556,0.444444,0.333333,0.333333,0.333333,0.666667,...,0.666667,0.666667,0.666667,0.555556,0.333333,0.555556,0.333333,0.555556,0.666667,0.444444
/title/tt0092099/,0.222222,0.444444,0.333333,0.333333,0.444444,0.555556,0.222222,0.333333,0.222222,0.555556,...,0.666667,0.666667,0.555556,0.333333,0.222222,0.333333,0.333333,0.333333,0.444444,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/title/tt0095188/,0.333333,0.555556,0.333333,0.333333,0.555556,0.333333,0.444444,0.222222,0.333333,0.555556,...,0.555556,0.555556,0.555556,0.555556,0.333333,0.444444,0.333333,0.555556,0.666667,0.444444
/title/tt0120630/,0.444444,0.444444,0.333333,0.444444,0.333333,0.333333,0.444444,0.333333,0.222222,0.444444,...,0.444444,0.444444,0.444444,0.333333,0.444444,0.333333,0.444444,0.333333,0.444444,0.444444
/title/tt0437863/,0.333333,0.777778,0.333333,0.444444,0.666667,0.333333,0.333333,0.333333,0.333333,0.555556,...,0.555556,0.555556,0.555556,0.555556,0.444444,0.333333,0.555556,0.555556,0.444444,0.666667
/title/tt0090583/,0.222222,0.444444,0.222222,0.222222,0.444444,0.333333,0.333333,0.111111,0.222222,0.444444,...,0.555556,0.555556,0.444444,0.333333,0.222222,0.222222,0.333333,0.333333,0.444444,0.333333


In [794]:
%%time
cos_feats_mat[titles].drop(titles)

CPU times: user 3.81 ms, sys: 368 µs, total: 4.18 ms
Wall time: 3.94 ms


,/title/tt7286456/,/title/tt0172495/,/title/tt0468569/,/title/tt0068646/,/title/tt0114709/
/title/tt0010323/,0.111111,0.111111,0.111111,0.111111,0.111111
/title/tt0012349/,0.444444,0.444444,0.444444,0.555556,0.444444
/title/tt0013442/,0.111111,0.111111,0.111111,0.222222,0.000000
/title/tt0015648/,0.111111,0.111111,0.111111,0.222222,0.111111
/title/tt0015864/,0.444444,0.444444,0.444444,0.555556,0.333333
...,...,...,...,...,...
/title/tt9845564/,0.222222,0.111111,0.111111,0.222222,0.000000
/title/tt9848626/,0.444444,0.333333,0.333333,0.333333,0.444444
/title/tt9860728/,0.444444,0.333333,0.333333,0.333333,0.222222
/title/tt9866072/,0.444444,0.333333,0.333333,0.333333,0.222222


In [796]:
cos_feats_mat[titles].drop(titles)

,/title/tt7286456/,/title/tt0172495/,/title/tt0468569/,/title/tt0068646/,/title/tt0114709/
/title/tt0010323/,0.111111,0.111111,0.111111,0.111111,0.111111
/title/tt0012349/,0.444444,0.444444,0.444444,0.555556,0.444444
/title/tt0013442/,0.111111,0.111111,0.111111,0.222222,0.000000
/title/tt0015648/,0.111111,0.111111,0.111111,0.222222,0.111111
/title/tt0015864/,0.444444,0.444444,0.444444,0.555556,0.333333
...,...,...,...,...,...
/title/tt9845564/,0.222222,0.111111,0.111111,0.222222,0.000000
/title/tt9848626/,0.444444,0.333333,0.333333,0.333333,0.444444
/title/tt9860728/,0.444444,0.333333,0.333333,0.333333,0.222222
/title/tt9866072/,0.444444,0.333333,0.333333,0.333333,0.222222


In [801]:
top_n

30

In [803]:
cos_feats_mat[titles[0]].nlargest(30)

/title/tt7286456/    1.000000
/title/tt0070047/    0.777778
/title/tt0133093/    0.777778
/title/tt0407887/    0.777778
/title/tt0448115/    0.777778
/title/tt1745960/    0.777778
/title/tt2179136/    0.777778
/title/tt3315342/    0.777778
/title/tt3741700/    0.777778
/title/tt6966692/    0.777778
/title/tt0040897/    0.666667
/title/tt0070735/    0.666667
/title/tt0073195/    0.666667
/title/tt0073486/    0.666667
/title/tt0076759/    0.666667
/title/tt0080684/    0.666667
/title/tt0086190/    0.666667
/title/tt0096895/    0.666667
/title/tt0097165/    0.666667
/title/tt0097576/    0.666667
/title/tt0100404/    0.666667
/title/tt0102926/    0.666667
/title/tt0103064/    0.666667
/title/tt0103776/    0.666667
/title/tt0106977/    0.666667
/title/tt0107290/    0.666667
/title/tt0109635/    0.666667
/title/tt0112462/    0.666667
/title/tt0114369/    0.666667
/title/tt0118688/    0.666667
Name: /title/tt7286456/, dtype: float64

In [815]:
%%time
candidate_titles = []
cos_feat_mat_ = cos_feats_mat[titles].drop(titles)
for title in titles:
    candidate_titles.append(cos_feat_mat_[title].nlargest(top_n))
top_titles = pd.concat(candidate_titles).groupby(level=0).last().nlargest(top_n)

CPU times: user 12.3 ms, sys: 5.53 ms, total: 17.8 ms
Wall time: 17.1 ms


In [817]:
top_titles.to_dict()

{'/title/tt0071562/': 0.8888888888888891,
 '/title/tt0372784/': 0.8888888888888891,
 '/title/tt0407887/': 0.8888888888888891,
 '/title/tt0816692/': 0.8888888888888891,
 '/title/tt1375666/': 0.8888888888888891,
 '/title/tt0031381/': 0.7777777777777779,
 '/title/tt0036868/': 0.7777777777777779,
 '/title/tt0044672/': 0.7777777777777779,
 '/title/tt0052618/': 0.7777777777777779,
 '/title/tt0059742/': 0.7777777777777779,
 '/title/tt0062622/': 0.7777777777777779,
 '/title/tt0066206/': 0.7777777777777779,
 '/title/tt0070047/': 0.7777777777777779,
 '/title/tt0070735/': 0.7777777777777779,
 '/title/tt0071315/': 0.7777777777777779,
 '/title/tt0072308/': 0.7777777777777779,
 '/title/tt0078788/': 0.7777777777777779,
 '/title/tt0082971/': 0.7777777777777779,
 '/title/tt0097576/': 0.7777777777777779,
 '/title/tt0113277/': 0.7777777777777779,
 '/title/tt0120737/': 0.7777777777777779,
 '/title/tt0120855/': 0.7777777777777779,
 '/title/tt0167260/': 0.7777777777777779,
 '/title/tt0167261/': 0.7777777777

In [766]:
cos_feats_mat

,/title/tt0010323/,/title/tt0012349/,/title/tt0013442/,/title/tt0015648/,/title/tt0015864/,/title/tt0016220/,/title/tt0017075/,/title/tt0017136/,/title/tt0018455/,/title/tt0018578/,...,/title/tt9701940/,/title/tt9701942/,/title/tt9777644/,/title/tt9783600/,/title/tt9784456/,/title/tt9845564/,/title/tt9848626/,/title/tt9860728/,/title/tt9866072/,/title/tt9898858/
/title/tt0010323/,1.000000,0.444444,0.666667,0.555556,0.333333,0.333333,0.333333,0.444444,0.222222,0.333333,...,0.333333,0.333333,0.333333,0.222222,0.222222,0.333333,0.222222,0.222222,0.333333,0.333333
/title/tt0012349/,0.444444,1.000000,0.444444,0.555556,0.888889,0.444444,0.333333,0.444444,0.444444,0.555556,...,0.555556,0.555556,0.555556,0.444444,0.333333,0.333333,0.444444,0.444444,0.444444,0.555556
/title/tt0013442/,0.666667,0.444444,1.000000,0.555556,0.555556,0.555556,0.444444,0.555556,0.333333,0.444444,...,0.444444,0.444444,0.444444,0.222222,0.222222,0.444444,0.111111,0.222222,0.333333,0.222222
/title/tt0015648/,0.555556,0.555556,0.555556,1.000000,0.444444,0.444444,0.333333,0.333333,0.333333,0.444444,...,0.444444,0.444444,0.444444,0.222222,0.222222,0.444444,0.222222,0.222222,0.333333,0.333333
/title/tt0015864/,0.333333,0.888889,0.555556,0.444444,1.000000,0.555556,0.444444,0.444444,0.555556,0.555556,...,0.555556,0.555556,0.555556,0.444444,0.333333,0.333333,0.333333,0.444444,0.444444,0.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/title/tt9845564/,0.333333,0.333333,0.444444,0.444444,0.333333,0.333333,0.333333,0.333333,0.222222,0.444444,...,0.666667,0.666667,0.666667,0.555556,0.444444,1.000000,0.333333,0.555556,0.666667,0.444444
/title/tt9848626/,0.222222,0.444444,0.111111,0.222222,0.333333,0.222222,0.111111,0.111111,0.222222,0.333333,...,0.555556,0.555556,0.555556,0.666667,0.555556,0.333333,1.000000,0.666667,0.555556,0.777778
/title/tt9860728/,0.222222,0.444444,0.222222,0.222222,0.444444,0.222222,0.222222,0.222222,0.333333,0.444444,...,0.666667,0.666667,0.666667,0.888889,0.777778,0.555556,0.666667,1.000000,0.888889,0.777778
/title/tt9866072/,0.333333,0.444444,0.333333,0.333333,0.444444,0.333333,0.333333,0.222222,0.333333,0.555556,...,0.777778,0.777778,0.777778,0.777778,0.666667,0.666667,0.555556,0.888889,1.000000,0.666667


In [705]:
cos_feats_mat.loc[selected_titles].T.nlargest(top_n + 1, '/title/tt0068646/')[1:]

,/title/tt0068646/
/title/tt0071562/,0.888889
/title/tt0816692/,0.888889
/title/tt0031381/,0.777778
/title/tt0036868/,0.777778
/title/tt0044672/,0.777778
/title/tt0052618/,0.777778
/title/tt0059742/,0.777778
/title/tt0062622/,0.777778
/title/tt0066206/,0.777778
/title/tt0070047/,0.777778


In [707]:
%%time
selected_titles = [
    '/title/tt0068646/',
    # '/title/tt0372784/',
    # '/title/tt0468569/'
]
top_n = 30
recommended_title_id = cos_feats_mat.loc[selected_titles].T.nlargest(top_n + 1, '/title/tt0068646/')[1:]
metadata_truncated.loc[recommended_title_id.index]

CPU times: user 2.9 ms, sys: 743 µs, total: 3.64 ms
Wall time: 3.3 ms


,main_genre,original_title,poster_url,actors,imdb_recommendations,reviews_collected_flg,rating,num_votes,user_review_num,critic_review_num,...,original_language,production_company_1,production_company_2,production_company_3,filming_location,filming_country,budget,boxoffice_gross_domestic,boxoffice_gross_opening,boxoffice_gross_worldwide
/title/tt0071562/,Crime,The Godfather: Part II,https://m.media-amazon.com/images/M/MV5BMWMwMG...,"{'1': '/name/nm0000199', '2': '/name/nm0000134...","{'1': '/title/tt0068646/', '2': '/title/tt0099...",1,9.0,1200000,1200.0,187.0,...,English,Paramount Pictures,The Coppola Company,American Zoetrope,"Kaiser Estate, 4000 W Lake Blvd, Homewood, Lak...",USA,"$13,000,000 (estimated)","$47,834,595","$171,417","$47,961,919"
/title/tt0816692/,Adventure,Interstellar,https://m.media-amazon.com/images/M/MV5BZjdkOT...,"{'1': '/name/nm0000190', '2': '/name/nm0004266...","{'1': '/title/tt1375666/', '2': '/title/tt0468...",1,8.6,1800000,5100.0,633.0,...,English,Paramount Pictures,Warner Bros.,Legendary Entertainment,Iceland,Iceland,"$165,000,000 (estimated)","$188,020,017","$47,510,360","$701,729,206"
/title/tt0031381/,Drama,Gone with the Wind,https://m.media-amazon.com/images/M/MV5BYjUyZW...,"{'1': '/name/nm0000022', '2': '/name/nm0000046...","{'1': '/title/tt0034583/', '2': '/title/tt0052...",1,8.2,312000,965.0,192.0,...,English,Selznick International Pictures,Metro-Goldwyn-Mayer (MGM),None,"North Little Rock, Arkansas, USA",USA,"$3,977,000 (estimated)","$200,882,193","$1,192,593","$402,382,193"
/title/tt0036868/,Drama,The Best Years of Our Lives,https://m.media-amazon.com/images/M/MV5BY2RmNT...,"{'1': '/name/nm0001485', '2': '/name/nm0000763...","{'1': '/title/tt0032551/', '2': '/title/tt0035...",1,8.1,64000,333.0,113.0,...,English,The Samuel Goldwyn Company,None,None,Ontario International Airport - 2900 E. Airpor...,USA,"$2,100,000 (estimated)","$23,650,000",None,"$23,656,620"
/title/tt0044672/,Drama,The Greatest Show on Earth,https://m.media-amazon.com/images/M/MV5BMzg5MW...,"{'1': '/name/nm0000071', '2': '/name/nm0000032...","{'1': '/title/tt0048960/', '2': '/title/tt0033...",1,6.6,15000,146.0,63.0,...,English,Paramount Pictures,None,None,"Sarasota, Florida, USA",USA,"$4,000,000 (estimated)","$36,000,000",None,"$36,000,000"
/title/tt0052618/,Adventure,Ben-Hur,https://m.media-amazon.com/images/M/MV5BNjgxY2...,"{'1': '/name/nm0000032', '2': '/name/nm0370144...","{'1': '/title/tt0056172/', '2': '/title/tt0031...",1,8.1,237000,493.0,134.0,...,English,Metro-Goldwyn-Mayer (MGM),None,None,Corner of Via di Salone and Via delle Case Ros...,Italy,"$15,000,000 (estimated)","$74,432,704","$241,792","$74,437,720"
/title/tt0059742/,Biography,The Sound of Music,https://m.media-amazon.com/images/M/MV5BODIxNj...,"{'1': '/name/nm0000267', '2': '/name/nm0001626...","{'1': '/title/tt0058331/', '2': '/title/tt0032...",1,8.1,229000,529.0,136.0,...,English,Robert Wise Productions,Argyle Enterprises,None,"Felsenreitschule, Salzburg, Austria",Austria,"$8,200,000 (estimated)","$159,287,539","$413,497","$159,428,329"
/title/tt0062622/,Adventure,2001: A Space Odyssey,https://m.media-amazon.com/images/M/MV5BMmNlYz...,"{'1': '/name/nm0001158', '2': '/name/nm0516972...","{'1': '/title/tt0066921/', '2': '/title/tt0081...",1,8.3,656000,2400.0,286.0,...,English,Metro-Goldwyn-Mayer (MGM),Stanley Kubrick Productions,None,"Isle of Harris, Western Isles, Scotland, UK",UK,"$12,000,000 (estimated)","$60,481,243","$202,759","$65,877,808"
/title/tt0066206/,Biography,Patton,https://m.media-amazon.com/images/M/MV5BMmNhZm...,"{'1': '/name/nm0001715', '2': '/name/nm0001500...","{'1': '/title/tt0066473/', '2': '/title/tt0067...",1,7.9,102000,330.0,120.0,...,English,Twentieth Century Fox,None,None,"Cabo de Gata, Almería, Andalucía, Spain",Spain,"$12,000,000 (estimated)","$61,749,765",None,"$61,749,765"
/title/tt0070047/,Horror,The Exorcist,https://m.media-amazon.com/images/M/MV5BYWFlZG...,"{'1': '/name/nm0000995', '2': '/name/nm0001884...","{'1': '/titl

In [ ]:
selected_titles = [
    '/title/tt0068646/',
    '/title/tt0372784/',
    '/title/tt0468569/'
]

In [531]:
metadata_truncated.query('original_title == "The Godfather"').index[0]

'/title/tt0068646/'

In [687]:
metadata_truncated.query('original_title == "Batman Begins"').index[0]

'/title/tt0372784/'

In [688]:
metadata_truncated.query('original_title == "The Dark Knight"').index[0]

'/title/tt0468569/'

In [532]:
target_title = metadata_truncated.query('original_title == "The Godfather"').index[0]
target_feats = feats.loc[target_title].values.reshape(1, -1)
target_feats.shape

(1, 136)